# Notebook for CCG Generation

## Defining a lexicon

In [1]:
class Lexicon:
    def __init__(self):
        # Dictionary to store lexical entries, where the key is a word
        # and the value is a list of categories (with optional semantics).
        self.entries = {}


    def add_entry(self, word, category, semantics=None):
        # Add a lexical entry to the lexicon.
        if word not in self.entries:
            self.entries[word] = []
        self.entries[word].append({'category': category, 'semantics': semantics})

    def remove_entry(self, word):
        # Remove a specific word from the lexicon.
        if word in self.entries:
            del self.entries[word]

    def get_categories(self, word):
        # Retrieve all categories for a given word.
        return self.entries.get(word, [])

    def __str__(self):
        # Display all entries in the lexicon.
        lexicon_str = "Lexicon:\n"
        for word, categories in self.entries.items():
            lexicon_str += f"{word}:\n"
            for entry in categories:
                semantics = entry['semantics'] if entry['semantics'] else 'None'
                lexicon_str += f"  - Category: {entry['category']}, Semantics: {semantics}\n"
        return lexicon_str
    
    def word_count(self):
        return len(self.entries) # Number of unique words

In [2]:
## EXAMPLE 

# Create an instance of the Lexicon
lexicon = Lexicon()

# Add entries
lexicon.add_entry("John", "NP", semantics="john")
lexicon.add_entry("Mary", "NP", semantics="mary")
lexicon.add_entry("Jack", "NP", semantics="jack")
lexicon.add_entry("Bill", "NP", semantics="bill")
lexicon.add_entry("likes", "(S\\NP)/NP", semantics="λx.λy.likes(y, x)")
lexicon.add_entry("runs", "S\\NP", semantics="λx.runs(x)")
lexicon.add_entry("the", "NP/N", semantics="λx.x")
lexicon.add_entry("dog", "N", semantics="dog")
lexicon.add_entry("cat", "N", semantics="cat")
lexicon.add_entry("brown", "N/N", semantics="λx.brown(x)")
lexicon.add_entry("big", "N/N", semantics="λx.big(x)")
lexicon.add_entry("and", "CONJ", semantics="λx.λy.and(x, y)")
lexicon.add_entry("would", "(S\\NP)/VP", semantics="λx.λy.will(y, x)")
lexicon.add_entry("prefer", "VP/NP", semantics="λx.prefer(x)")

# Handle ambiguous entries (e.g., "saw" as both a verb and noun)
lexicon.add_entry("saw", "(S\\NP)/NP", semantics="λx.λy.saw(y, x)")
lexicon.add_entry("saw", "N", semantics="saw")

In [3]:
## EXAMPLE 

# Retrieve categories for a specific word
word = "likes"
print(f"Categories for '{word}':")
for entry in lexicon.get_categories(word):
    print(f"  - Category: {entry['category']}, Semantics: {entry['semantics']}")

# Print the entire lexicon
print(lexicon)

print(f"\nTotal number of unique words in the lexicon: {lexicon.word_count()}")

Categories for 'likes':
  - Category: (S\NP)/NP, Semantics: λx.λy.likes(y, x)
Lexicon:
John:
  - Category: NP, Semantics: john
Mary:
  - Category: NP, Semantics: mary
Jack:
  - Category: NP, Semantics: jack
Bill:
  - Category: NP, Semantics: bill
likes:
  - Category: (S\NP)/NP, Semantics: λx.λy.likes(y, x)
runs:
  - Category: S\NP, Semantics: λx.runs(x)
the:
  - Category: NP/N, Semantics: λx.x
dog:
  - Category: N, Semantics: dog
cat:
  - Category: N, Semantics: cat
brown:
  - Category: N/N, Semantics: λx.brown(x)
big:
  - Category: N/N, Semantics: λx.big(x)
and:
  - Category: CONJ, Semantics: λx.λy.and(x, y)
would:
  - Category: (S\NP)/VP, Semantics: λx.λy.will(y, x)
prefer:
  - Category: VP/NP, Semantics: λx.prefer(x)
saw:
  - Category: (S\NP)/NP, Semantics: λx.λy.saw(y, x)
  - Category: N, Semantics: saw


Total number of unique words in the lexicon: 15


## Generating sentences (including trees) without semantic constraints

In [4]:
import random

class SentenceGenerator: 
    #A class is a blueprint for creating objects (instances), 
    # which can hold data and have functions (methods) associated with them.
    def __init__(self, lexicon): 
        # Constructor method for the class: special method in Python automatically called when an 
        # object (an instance of the class) is created.
        self.lexicon = lexicon 
        # This line stores the lexicon parameter passed to the constructor as an instance attribute.

    def generate_sentence(self, target_category="S"):
        # Generate a sentence that matches the target category (default is 'S' for a sentence).
        # Defines a method named generate_sentence: 
        # a member function of the SentenceGenerator class, 
        # meaning it operates on instances of that class.
        sentence, remaining_category = self.build_sentence(target_category) 
        # This line calls the build_sentence method: result is expected to be a tuple.
        
        # If we couldn't completely match the category, return None
        if remaining_category is None: #This part handles the outcome of the sentence generation.
            return " ".join(sentence)
        else:
            return None

    def generate_multiple_sentences(self, target_category="S", count=5):
        # Generate multiple unique sentences that match the target category.
        sentences = set() #This is a set that will hold the unique sentences generated.
        attempts = 0 #This counter keeps track of the number of attempts made to generate a sentence.
        max_attempts = count * 5  # Allow multiple attempts to find unique sentences
        
        # Ensures loop continues until desired number of unique sentences has been generated or 
        # max number of attempts is reached.
        while len(sentences) < count and attempts < max_attempts:
            attempts += 1
            sentence = self.generate_sentence(target_category)
            if sentence and sentence not in sentences:
                sentences.add(sentence)
        
        return list(sentences)

    def build_sentence(self, target_category, depth=0, max_depth=10, used_conjunction=False, used_adjectives=None):
        # Initialize the set to track used adjectives if it's not passed in
        if used_adjectives is None:
            used_adjectives = set()

        if depth > max_depth:
            print(f"Exceeded max recursion depth: {depth} for target {target_category}")
            return [], None, None  # Include tree in return values


        if target_category == "S":
            # Randomly decide decomposition type with a 50/50 chance
            if random.choice([True, False]):
                # Decompose S into NP and S\NP
                print("Decomposing S into NP and S\\NP")
                np_sentence, np_category, np_tree = self.build_sentence("NP", depth + 1, max_depth, used_conjunction, used_adjectives)
                if np_category is None:  # NP found
                    print(f"NP successfully resolved: {np_sentence}")
                    snp_sentence, snp_category, snp_tree = self.build_sentence("S\\NP", depth + 1, max_depth, used_conjunction, used_adjectives)
                    if snp_category is None:  # S\NP found
                        print(f"Successfully decomposed S: NP='{np_sentence}' and S\\NP='{snp_sentence}'")
                        tree = {
                            "node": "S",
                            "children": [np_tree, snp_tree]
                        }
                        return np_sentence + snp_sentence, None, tree
                    else:
                        print(f"Failed to resolve S\\NP after matching NP='{np_sentence}'")
                else:
                    print(f"Failed to resolve NP for S decomposition")
            else:
                # Decompose S into S/NP and NP
                print("Decomposing S into S/NP and NP")
                snp_sentence, snp_category, snp_tree = self.build_sentence("S/NP", depth + 1, max_depth, used_conjunction, used_adjectives)
                if snp_category is None:  # S/NP found
                    print(f"S/NP successfully resolved: {snp_sentence}")
                    np_sentence, np_category, np_tree = self.build_sentence("NP", depth + 1, max_depth, used_conjunction, used_adjectives)
                    if np_category is None:  # NP found
                        print(f"Successfully decomposed S: S/NP='{snp_sentence}' and NP='{np_sentence}'")
                        tree = {
                            "node": "S",
                            "children": [snp_tree, np_tree]
                        }
                        return snp_sentence + np_sentence, None, tree
                    else:
                        print(f"Failed to resolve NP after matching S/NP='{snp_sentence}'")
                else:
                    print(f"Failed to resolve S/NP for S decomposition")
        
        if target_category == "S:DCL":
            # Randomly decide decomposition type with a 50/50 chance
            if random.choice([True, False]):
                # Decompose S:DCL into NP and S:DCL\NP
                print("Decomposing S:DCL into NP and S:DCL\\NP")
                np_sentence, np_category, np_tree = self.build_sentence("NP", depth + 1, max_depth, used_conjunction, used_adjectives)
                if np_category is None:  # NP found
                    print(f"NP successfully resolved: {np_sentence}")
                    snp_sentence, snp_category, snp_tree = self.build_sentence("S:DCL\\NP", depth + 1, max_depth, used_conjunction, used_adjectives)
                    if snp_category is None:  # S:DCL\NP found
                        print(f"Successfully decomposed S:DCL: NP='{np_sentence}' and S:DCL\\NP='{snp_sentence}'")
                        tree = {
                            "node": "S:DCL",
                            "children": [np_tree, snp_tree]
                        }
                        return np_sentence + snp_sentence, None, tree
                    else:
                        print(f"Failed to resolve S:DCL\\NP after matching NP='{np_sentence}'")
                else:
                    print(f"Failed to resolve NP for S:DCL decomposition")
            else:
                # Decompose S:DCL into S:DCL/NP and NP
                print("Decomposing S:DCL into S:DCL/NP and NP")
                snp_sentence, snp_category, snp_tree = self.build_sentence("S:DCL/NP", depth + 1, max_depth, used_conjunction, used_adjectives)
                if snp_category is None:  # S:DCL/NP found
                    print(f"S:DCL/NP successfully resolved: {snp_sentence}")
                    np_sentence, np_category, np_tree = self.build_sentence("NP", depth + 1, max_depth, used_conjunction, used_adjectives)
                    if np_category is None:  # NP found
                        print(f"Successfully decomposed S:DCL: S:DCL/NP='{snp_sentence}' and NP='{np_sentence}'")
                        tree = {
                            "node": "S:DCL",
                            "children": [snp_tree, np_tree]
                        }
                        return snp_sentence + np_sentence, None, tree
                    else:
                        print(f"Failed to resolve NP after matching S:DCL/NP='{snp_sentence}'")
                else:
                    print(f"Failed to resolve S:DCL/NP for S:DCL decomposition")
        

        lexicon_keys = list(self.lexicon.entries.keys())
        random.shuffle(lexicon_keys)

        for word in lexicon_keys:
            entries = self.lexicon.entries[word]
            for entry in entries:
                category = entry['category']
                print(f"Trying word '{word}' with category '{category}' to match '{target_category}'")

                # Skip repeated adjectives
                if category == "N/N" and word in used_adjectives:
                    print(f"Skipping repeated adjective '{word}'")
                    continue

                # Direct match:
                if self.normalize_category(category) == self.normalize_category(target_category):
                    print(f"Direct match found: '{word}' for category '{target_category}'")
                    if category == "N/N":
                        used_adjectives.add(word)  # Mark this adjective as used
                    tree = {"node": target_category, "children": [word]}
                    return [word], None, tree

                # Conjunction
                if category == "CONJ" and target_category in ["NP", "S", "(S\\NP)/NP", "S\\NP"] and not used_conjunction:
                    print(f"Trying conjunction '{word}' for category '{target_category}'")
                    left_sentence, remaining_category, left_tree = self.build_sentence(target_category, depth + 1, max_depth, True, used_adjectives)
                    if remaining_category is None:
                        print(f"Conjunction partly successful for '{word}'")
                        right_sentence, remaining_category, right_tree = self.build_sentence(target_category, depth + 1, max_depth, True, used_adjectives)
                        if remaining_category is None:
                            print(f"Conjunction successful for '{word}'")
                            tree = {
                                "node": target_category,
                                "children": [left_tree, {"node": "CONJ", "children": [word]}, right_tree]
                            }
                            return left_sentence + [word] + right_sentence, None, tree

                # Forward application: (A/B) + B → A
                if self.is_forward_function(category):
                    left_category, right_category = map(self.normalize_category, self.split_category(category))
                    if self.normalize_category(target_category) == left_category:
                        print(f"Attempting forward application: '{word}' as ({left_category}/{right_category})")
                        if category == "N/N":
                            used_adjectives.add(word)  # Mark this adjective as used
                        right_sentence, remaining_category, right_tree = self.build_sentence(right_category, depth + 1, max_depth, used_conjunction, used_adjectives)
                        if remaining_category is None:
                            print(f"Forward application successful for '{word}'")
                            tree = {
                                "node": left_category,
                                "children": [{"node": f"({left_category}/{right_category})", "children": [word]}, right_tree]
                            }
                            return [word] + right_sentence, None, tree

                # Backward application: B + (A\B) → A
                if self.is_backward_function(category):
                    left_category, right_category = map(self.normalize_category, self.split_category(category))
                    if self.normalize_category(target_category) == left_category:
                        print(f"Attempting backward application: '{word}' as ({left_category}\\{right_category})")
                        left_sentence, remaining_category, left_tree = self.build_sentence(right_category, depth + 1, max_depth, used_conjunction, used_adjectives)
                        if remaining_category is None:
                            print(f"Backward application successful for '{word}'")
                            tree = {
                                "node": left_category,
                                "children": [left_tree, {"node": f"({left_category}\\{right_category})", "children": [word]}]
                            }
                            return left_sentence + [word], None, tree
                        
                # Type raising and forward composition in one for NP (→ S/(S\NP)) + (S\NP)/NP → S/NP
                # for declarative sentences
                if target_category == "S:DCL/NP" and category == "NP":
                    print(f"Attempting type raising and forward composition for '{word}' to obtain S:DCL/NP")
                    right_sentence, remaining_category, right_tree = self.build_sentence("(S:DCL\\NP)/NP", depth + 1, max_depth, used_conjunction, used_adjectives)
                    if remaining_category is None:
                        print(f"Type raising and forward composition successful for '{word}'")
                        tree = {
                            "node": "S:DCL/NP",
                            "children": [{"node": "NP", "children": [word]}, right_tree]
                        }
                        return [word] + right_sentence, None, tree
                
                # # Forward composition: (A/B) + (B/C) → A/C (not necessary?)
                # if self.is_forward_function(category) and self.is_forward_function(target_category):
                #     left_category, right_category = map(self.normalize_category, self.split_category(category))
                #     left_target_category, right_target_category = self.split_category(target_category)
                #     if left_target_category == left_category:
                #         print(f"Attempting forward composition: '{word}' as ({left_category}/{right_category})")
                #         right_sentence, remaining_category, right_tree = self.build_sentence(f"{right_category}/{right_target_category}", depth + 1, max_depth, used_conjunction, used_adjectives)
                #         if remaining_category is None:
                #             print(f"Forward composition successful for '{word}'")
                #             tree = {
                #                 "node": target_category,
                #                 "children": [{"node": f"({left_category}/{right_category})", "children": [word]}, right_tree]
                #             }
                #             return [word] + right_sentence, None, tree
                #     elif right_target_category == right_category:
                #         print(f"Attempting forward composition: '{word}' as ({left_category}/{right_category})")
                #         left_sentence, remaining_category, left_tree = self.build_sentence(f"{left_target_category}/{left_category}", depth + 1, max_depth, used_conjunction, used_adjectives)
                #         if remaining_category is None:
                #             print(f"Forward composition successful for '{word}'")
                #             tree = {
                #                 "node": target_category,
                #                 "children": [left_tree, {"node": f"({left_category}/{right_category})", "children": [word]}]
                #             }
                #             return left_sentence + [word], None, tree
                        
                # # Backward composition: (B\C) + (A\B) → A\C (not necessary?)
                # if self.is_backward_function(category) and self.is_backward_function(target_category):
                #     left_category, right_category = map(self.normalize_category, self.split_category(category))
                #     left_target_category, right_target_category = self.split_category(target_category)
                #     if left_target_category == left_category:
                #         print(f"Attempting backward composition: '{word}' as ({left_category}\\{right_category})")
                #         left_sentence, remaining_category, left_tree = self.build_sentence(f"{right_category}\\{right_target_category}", depth + 1, max_depth, used_conjunction, used_adjectives)
                #         if remaining_category is None:
                #             print(f"Backward composition successful for '{word}'")
                #             tree = {
                #                 "node": target_category,
                #                 "children": [left_tree, {"node": f"({left_category}/{right_category})", "children": [word]}]
                #             }
                #             return left_sentence + [word], None, tree
                #     elif right_target_category == right_category:
                #         print(f"Attempting backward composition: '{word}' as ({left_category}\\{right_category})")
                #         right_sentence, remaining_category, right_tree = self.build_sentence(f"{left_target_category}\\{left_category}", depth + 1, max_depth, used_conjunction, used_adjectives)
                #         if remaining_category is None:
                #             print(f"Backward composition successful for '{word}'")
                #             tree = {
                #                 "node": target_category,
                #                 "children": [{"node": f"({left_category}/{right_category})", "children": [word]}, right_tree]
                #             }
                #             return [word] + right_sentence, None, tree


        print(f"No match found for target '{target_category}' at depth {depth}")
        return [], target_category, None


    def is_forward_function(self, category):
        # Check if the category is a forward function (A/B).
        return "/" in category

    def is_backward_function(self, category):
        # Check if the category is a backward function (A\B).
        return "\\" in category

    def split_category(self, function_category):
        # Split a function category (A/B or A\B) into its components A and B.
        if "/" in function_category:
            return function_category.split("/", 1)
        elif "\\" in function_category:
            return function_category.split("\\", 1)
        return None, None
    
    def normalize_category(self, category):
        # Remove unnecessary parentheses from a category.
        while category.startswith("(") and category.endswith(")"):
            category = category[1:-1]
        return category

In [5]:
import anytree
from anytree import Node, RenderTree

# New code for visualizing the tree
def build_visual_tree(tree, parent=None):
    """
    Recursively convert the JSON-like tree into an anytree.Node tree.
    """
    if isinstance(tree, dict):
        node = Node(tree['node'], parent=parent)
        for child in tree.get('children', []):
            build_visual_tree(child, parent=node)
        return node
    elif isinstance(tree, str):  # Leaf node (word)
        return Node(tree, parent=parent)

In [6]:
## EXAMPLE 

import json

generator = SentenceGenerator(lexicon)

# Generate multiple sentences and store them in a matrix
num_sentences = 5  # Adjust the number of sentences as needed
matrix = []  # Initialize the matrix to store sentences and trees

# Generate sentences and trees
for _ in range(num_sentences):
    sentence, _, tree = generator.build_sentence("S")  # Generate sentence and its tree
    matrix.append([" ".join(sentence), tree])  # Append as a row in the matrix

# Print the matrix with ASCII tree visualizations
print("Generated Sentences and Trees:")
for i, (sentence, tree) in enumerate(matrix):
    print(f"Sentence {i + 1}: {sentence}")
    ascii_tree_root = build_visual_tree(tree)  # Convert tree to anytree format
    print("Tree Visualization:")
    for pre, fill, node in RenderTree(ascii_tree_root):
        print(f"{pre}{node.name}")
    print("-" * 50)

# Save the matrix to a JSON file
with open("sentences_and_trees.json", "w") as f:
    json.dump(matrix, f, indent=2)

print("\nMatrix saved to 'sentences_and_trees.json'")

Decomposing S into S/NP and NP
Trying word 'Bill' with category 'NP' to match 'S/NP'
Trying word 'would' with category '(S\NP)/VP' to match 'S/NP'
Trying word 'likes' with category '(S\NP)/NP' to match 'S/NP'
Trying word 'brown' with category 'N/N' to match 'S/NP'
Trying word 'Mary' with category 'NP' to match 'S/NP'
Trying word 'saw' with category '(S\NP)/NP' to match 'S/NP'
Trying word 'saw' with category 'N' to match 'S/NP'
Trying word 'Jack' with category 'NP' to match 'S/NP'
Trying word 'runs' with category 'S\NP' to match 'S/NP'
Trying word 'prefer' with category 'VP/NP' to match 'S/NP'
Trying word 'John' with category 'NP' to match 'S/NP'
Trying word 'and' with category 'CONJ' to match 'S/NP'
Trying word 'the' with category 'NP/N' to match 'S/NP'
Trying word 'dog' with category 'N' to match 'S/NP'
Trying word 'big' with category 'N/N' to match 'S/NP'
Trying word 'cat' with category 'N' to match 'S/NP'
No match found for target 'S/NP' at depth 1
Failed to resolve S/NP for S decom

## Extracting lexicon from data and generating sentences

In [7]:
import json
import os
import re

# Function to normalize categories
def normalize_category(category):
    """
    Normalize syntactic categories:
    - Replace S:DCL with S, preserving structure in compound categories.
    """
    # Replace all occurrences of S:DCL with S, even in nested categories
    normalized = re.sub(r"\bS:DCL\b", "S", category)
    return normalized

# Function to extract words and categories from multiple files and populate the Lexicon
def populate_lexicon_from_folder(folder_path, lexicon):
    """
    Extract words and syntactic categories from all relevant files in a folder and populate the lexicon.
    Ensures that syntactic types (categories) are capitalized and S:DCL is treated as S.
    """
    pattern = re.compile(r"t\(([^,]+),\s*'([^']+)',\s*\[.*?\]\)")

    # List all relevant files in the folder
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith("en.parse.tags"):  # Process only files with this extension
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")
                with open(file_path, 'r') as f:
                    for line in f:
                        matches = pattern.findall(line)
                        for match in matches:
                            syntactic_type, word = match
                            # Capitalize the syntactic type and normalize S:DCL to S
                            #normalized_type = normalize_category(syntactic_type.upper())
                            lexicon.add_entry(word, syntactic_type.upper())
                            #print(f"Added entry: Word='{word}', Category='{normalized_type}'")

# Example usage

# Create an instance of Lexicon
lexicon2 = Lexicon()

# Path to the p00, p01, p02 folders
## Update the paths to the p00, p01, p02 folders on your system!
folder_path00 = "/Users/rubenreijerse/repos/ccg-lm/p00"
folder_path01 = "/Users/rubenreijerse/repos/ccg-lm/p01"
folder_path02 = "/Users/rubenreijerse/repos/ccg-lm/p02"

# Populate the lexicon from all files in the folder
populate_lexicon_from_folder(folder_path00, lexicon2)
populate_lexicon_from_folder(folder_path01, lexicon2)
populate_lexicon_from_folder(folder_path02, lexicon2)

lexicon2.remove_entry("\\\"")
lexicon2.remove_entry(",")
lexicon2.remove_entry(".")

# Print the populated lexicon
print(lexicon2)

# Print the total number of unique words in the lexicon
print(f"\nTotal number of unique words in the lexicon: {lexicon2.word_count()}")


Processing file: /Users/rubenreijerse/repos/ccg-lm/p00/d2508/en.parse.tags
Processing file: /Users/rubenreijerse/repos/ccg-lm/p00/d2153/en.parse.tags
Processing file: /Users/rubenreijerse/repos/ccg-lm/p00/d1948/en.parse.tags
Processing file: /Users/rubenreijerse/repos/ccg-lm/p00/d0856/en.parse.tags
Processing file: /Users/rubenreijerse/repos/ccg-lm/p00/d0869/en.parse.tags
Processing file: /Users/rubenreijerse/repos/ccg-lm/p00/d0055/en.parse.tags
Processing file: /Users/rubenreijerse/repos/ccg-lm/p00/d3420/en.parse.tags
Processing file: /Users/rubenreijerse/repos/ccg-lm/p00/d1383/en.parse.tags
Processing file: /Users/rubenreijerse/repos/ccg-lm/p00/d0867/en.parse.tags
Processing file: /Users/rubenreijerse/repos/ccg-lm/p00/d3418/en.parse.tags
Processing file: /Users/rubenreijerse/repos/ccg-lm/p00/d3285/en.parse.tags
Processing file: /Users/rubenreijerse/repos/ccg-lm/p00/d0858/en.parse.tags
Processing file: /Users/rubenreijerse/repos/ccg-lm/p00/d2303/en.parse.tags
Processing file: /Users/r

In [8]:
#Generate one sentence

generator2 = SentenceGenerator(lexicon2)

sentence2, _, tree2 = generator2.build_sentence("S:DCL")

sentence2join = " ".join(sentence2)
print(f"Sentence: {sentence2join}")
ascii_tree_root = build_visual_tree(tree2)  # Convert tree to anytree format
print("Tree Visualization:")
for pre, fill, node in RenderTree(ascii_tree_root):
    print(f"{pre}{node.name}")

Decomposing S:DCL into NP and S:DCL\NP
Trying word 'wedding' with category 'N' to match 'NP'
Trying word 'stolen' with category 'S:PSS\NP' to match 'NP'
Trying word 'stolen' with category 'S:PSS\NP' to match 'NP'
Trying word 'stolen' with category '(S:PT\NP)/NP' to match 'NP'
Trying word 'lead' with category '(S:DCL\NP)/PP' to match 'NP'
Trying word 'lead' with category 'N' to match 'NP'
Trying word 'prefer' with category '(S:B\NP)/NP' to match 'NP'
Trying word 'divorced' with category '(S:DCL\NP)/NP' to match 'NP'
Trying word 'creative' with category 'S:ADJ\NP' to match 'NP'
Trying word 'airport' with category 'N' to match 'NP'
Trying word 'slew' with category '(S:DCL\NP)/NP' to match 'NP'
Trying word 'minutes' with category 'N' to match 'NP'
Trying word 'minutes' with category 'N' to match 'NP'
Trying word 'too' with category '((S:ADJ\NP)/(S:TO\NP))/(S:ADJ\NP)' to match 'NP'
Trying word 'too' with category '(S\NP)\(S\NP)' to match 'NP'
Trying word 'too' with category 'S:DCL\S:DCL' to

In [9]:
## FOR GENERATING MANY SENTENCES FROM EXTRACTED LEXICON

from concurrent.futures import ThreadPoolExecutor
import json
from anytree import RenderTree

# Function to generate sentences in parallel
def generate_sentence_and_tree(generator, unique_sentences):
    while True:  # Loop until a unique sentence is generated
        sentence, _, tree = generator.build_sentence("S:DCL")
        sentence_str = " ".join(sentence)
        
        # Ensure the sentence ends with a dot
        if not sentence_str.endswith("."):
            sentence_str += "."
        
        # Check if the sentence is unique
        if sentence_str not in unique_sentences:
            return sentence_str, tree

# Main function to generate unique sentences
def generate_unique_sentences(generator, num_sentences, max_workers=4):
    unique_sentences = set()  # Set to store unique sentences
    matrix = []  # Matrix to store sentence-tree pairs
    duplicate_count = 0  # Counter for duplicates

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        while len(matrix) < num_sentences:
            futures = [
                executor.submit(generate_sentence_and_tree, generator, unique_sentences)
                for _ in range(max_workers)
            ]

            for future in futures:
                try:
                    sentence_str, tree = future.result()
                    if sentence_str not in unique_sentences:
                        unique_sentences.add(sentence_str)
                        matrix.append([sentence_str, tree])
                    else:
                        duplicate_count += 1
                except Exception as e:
                    print(f"Error during sentence generation: {e}")

    return matrix, duplicate_count

# Visualization and Saving
def visualize_and_save(matrix, output_file):
    print("Generated Unique Sentences and Trees:")
    for i, (sentence, tree) in enumerate(matrix):
        print(f"Sentence {i + 1}: {sentence}")
        ascii_tree_root = build_visual_tree(tree) 
        print("Tree Visualization:")
        for pre, fill, node in RenderTree(ascii_tree_root):
            print(f"{pre}{node.name}")
        print("-" * 50)

    # Save to a JSON file
    with open(output_file, "w") as f:
        json.dump(matrix, f, indent=2)

    print(f"\nMatrix saved to '{output_file}'")

# Usage
if __name__ == "__main__":
    generator2 = SentenceGenerator(lexicon2)
    num_sentences2 = 100
    matrix2, duplicate_count = generate_unique_sentences(generator2, num_sentences2, max_workers=4)
    visualize_and_save(matrix2, "sentences_and_trees2.json")

    print(f"\nNumber of duplicate sentences generated: {duplicate_count}")


Decomposing S:DCL into NP and S:DCL\NP
Trying word 'twice' with category '(S\NP)\(S\NP)' to match 'NP'Decomposing S:DCL into NP and S:DCL\NP
Decomposing S:DCL into NP and S:DCL\NPTrying word 'twice' with category '(S\NP)\(S\NP)' to match 'NP'

Trying word 'twice' with category '(S\NP)\(S\NP)' to match 'NP'
Trying word 'insulted' with category '(S:DCL\NP)/NP' to match 'NP'
Trying word 'trip' with category 'N' to match 'NP'Trying word 'crying' with category 'S:NG\NP' to match 'NP'

Decomposing S:DCL into NP and S:DCL\NPTrying word 'running' with category 'S:NG\NP' to match 'NP'Trying word 'trip' with category 'N' to match 'NP'

Trying word 'running' with category 'S:NG\NP' to match 'NP'
Trying word 'drought' with category 'N' to match 'NP'
Trying word 'disrespectful' with category 'S:ADJ\NP' to match 'NP'
Trying word 'am' with category '(S:DCL\NP)/(S:NG\NP)' to match 'NP'
Trying word 'am' with category '(S:DCL\NP)/(S:NG\NP)' to match 'NP'
Trying word 'am' with category '(S:DCL\NP)/PP' to